In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
final_data=pd.read_csv('Dataset/Final_Data_of_Attack_all_data.csv')
all_data = final_data

In [ ]:
# drop unuse data
all_data = all_data.drop(['attack', 'attack_types'], axis=1)

In [ ]:
# Split Data
feature = all_data[all_data.columns.difference(['attack_flag'])]
target= all_data[['attack_flag']]

In [ ]:
feature

,count,diff_srv_rate,dst_bytes,dst_host_count,dst_host_diff_srv_rate,dst_host_rerror_rate,dst_host_same_src_port_rate,dst_host_same_srv_rate,dst_host_serror_rate,dst_host_srv_count,...,service,src_bytes,srv_count,srv_diff_host_rate,srv_error_rate,srv_rerror_rate,su_attempted,urgent,wrong_fragment,duration
0,0.001,0.0,0.02957,0.001,0.0,0.000,0.100,0.1,0.000,0.003,...,0.05,0.01511,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0184
1,0.001,0.0,0.02766,0.002,0.0,0.000,0.050,0.1,0.000,0.004,...,0.05,0.01735,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0305
2,0.001,0.0,0.06707,0.001,0.0,0.000,0.100,0.1,0.000,0.001,...,0.05,0.01587,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0150
3,0.001,0.0,0.04551,0.001,0.0,0.000,0.100,0.1,0.000,0.001,...,0.05,0.02328,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0060
4,0.001,0.0,0.03095,0.002,0.0,0.000,0.050,0.1,0.000,0.002,...,0.05,0.01567,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817545,0.004,0.0,0.08314,0.096,0.0,0.001,0.001,0.1,0.001,0.096,...,0.00,0.54540,0.004,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
817546,0.004,0.0,0.08314,0.097,0.0,0.001,0.001,0.1,0.001,0.097,...,0.00,0.54540,0.004,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
817547,0.003,0.0,0.08314,0.098,0.0,0.001,0.001,0.1,0.001,0.098,...,0.00,0.54540,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
817548,0.003,0.0,0.08314,0.099,0.0,0.001,0.001,0.1,0.001,0.099,...,0.00,0.54540,0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0000


In [ ]:
sc = StandardScaler()
sc = sc.fit(feature)
feature = pd.DataFrame(sc.transform(feature), columns=feature.columns)

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(feature, target, test_size=0.2, random_state=102)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model_ababoost = AdaBoostClassifier()
model_ababoost.fit(X_train, y_train)
pred = model_ababoost.predict(X_test)
acc_ada = accuracy_score(pred, y_test)

print('Accuracy Score of adaBoost Classifier is : ', acc_ada)

c:\Users\sreym\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy Score of adaBoost Classifier is :  0.9991804782582105


In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_DT = DecisionTreeClassifier()
model_DT.fit(X_train, y_train)
pred_DT = model_DT.predict(X_test)
acc_DT = accuracy_score(pred_DT, y_test)

print('Accuracy Score of Decision Tree is : ', acc_DT)

Accuracy Score of Decision Tree is :  0.9998348724848634


In [ ]:
models = pd.DataFrame({
    'Model' : ['Decision Tree Classifier', 'Ada Boost Classifier'],
    'Score' : [acc_ada, acc_DT]
})

models.sort_values(by= 'Score', ascending= False, ignore_index=True)

,Model,Score
0,Ada Boost Classifier,0.999835
1,Decision Tree Classifier,0.999180


In [ ]:
# Predicted probabilities on testing data
X_test['test_pred'] = model_DT.predict(X_test)

In [ ]:
# Doing the final predictions
X_test['attack_flag'] = X_test['test_pred'].apply(lambda x: 1 if x>0.27 else 0)

In [ ]:
X_test['attack_flag'].value_counts()


attack_flag
0    115458
1     48052
Name: count, dtype: int64

Mutinomial classification

In [ ]:
multi_data = final_data
multi_data.drop(columns=['attack_flag'], inplace=True)

In [ ]:
multi_data['attack_types'] = multi_data['attack_types'].map({'normal':'normal', 'neptune':'dos', 'satan':'probe', 'smurf':'dos',
                                                'portsweep':'probe', 'nmap':'probe', 'back':'dos', 'guess_passwd':'r2l',
                                                'buffer_overflow':'u2r', 'rootkit':'u2r', 'ftp_write':'r2l'})


In [ ]:
multi_data['attack_types'].value_counts()

attack_types
normal    576710
dos       231203
probe       9537
r2l           60
u2r           40
Name: count, dtype: int64

In [ ]:
multi_data['attack_types'] = multi_data['attack_types'].map({'normal' : 0, 'dos': 1, 'probe':2, 'r2l': 3, 'u2r':4})

In [ ]:
# drop column
multi_data = multi_data.drop(['attack'], axis=1)

In [ ]:
pip install xgboost


     ---------------------------------------- 70.9/70.9 MB 1.0 MB/s eta 0:00:00


You should consider upgrading via the 'c:\Users\sreym\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
# Split Data
feature_multi = multi_data[multi_data.columns.difference(['attack_types'])]
target_multi= multi_data[['attack_types']]

In [ ]:
sc=StandardScaler()
sc=sc.fit(feature_multi)
X2=pd.DataFrame(sc.transform(feature_multi), columns=feature_multi.columns)

In [ ]:
train_X,test_X,train_y, test_y=train_test_split(feature_multi,target_multi, test_size=0.2, random_state=102)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression()
lr.fit(train_X,train_y)
pred_lr=lr.predict(test_X)
acc_lr = accuracy_score(pred_lr,test_y)
print('Accuracy Score of Logistic Regression is',acc_lr)

c:\Users\sreym\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy Score of Logistic Regression is 0.9951073328848389


c:\Users\sreym\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(train_X,train_y )
pred_xgb = xgb.predict(test_X)
acc_xgb=accuracy_score(pred_xgb,test_y)

print("Accuracy Score of XGB Classifier is : ",acc_xgb)

Accuracy Score of XGB Classifier is :  0.9998960308238029
